# Load packages

In [449]:
import dacy
import os
import copy
import re
import glob
import pandas as pd
from collections import Counter
from spacy.tokens import DocBin, Doc, Span
from spacy.training.corpus import Corpus


# Defining functions


In [450]:
# Defining a function for retriveing all ents for a given doc
def retrieve_all_ents(doc, all_docs):
    ents_for_doc = []
    # Acquire list of all ents for a doc
    for i in flat_list:
        if i.text == doc.text:
            for ent in i.ents:
                ents_for_doc.append(ent)
    return ents_for_doc


# Defining a function for exploding a doc and exploding its ents
def explode_doc(doc, ents):
    ents_exploded = [
        {
            "ent": ent,
            "ent.text": ent.text,
            "ent.label_": ent.label_,
            "ent.label_and_text": ent.text + ent.label_,
        }
        for ent in ents
    ]
    return {
        "doc.text": doc.text,
        "doc": doc,
        "doc.ents": ents_exploded,
    }


# Defining function for retrieving a list with unique ents, and the count of the unique ents
def doc_ents_count(exploded_doc, match):
    unique_ents = []
    unique_ents_count = []
    
    if match == 'label_and_text':
        for ent_idx in range(len(exploded_doc["doc.ents"])):
            ent = exploded_doc["doc.ents"][ent_idx]
            # If ent is in unique_ents, unique_ents_count += 1, for same index:
            if any(
                ent["ent.label_and_text"] == unique_ent["ent.label_and_text"]
                for unique_ent in unique_ents
            ):
                # print(f"""ent "{ent["ent.text"]}" already in unique_ents""")
                unique_ent_label_and_texts = [
                    unique_ent["ent.label_and_text"] for unique_ent in unique_ents
                ]
                index_of_same_ent = unique_ent_label_and_texts.index(
                    ent["ent.label_and_text"]
                )
                unique_ents_count[index_of_same_ent] += 1
            else:
                # print(f"""ent "{ent["ent.text"]}" not already in unique_ents""")
                unique_ents.append(ent)
                unique_ents_count.append(1)
    
    else:
        for ent_idx in range(len(exploded_doc["doc.ents"])):
            ent = exploded_doc["doc.ents"][ent_idx]
            # If ent is in unique_ents, unique_ents_count += 1, for same index:
            if any(
                ent["ent.text"] == unique_ent["ent.text"]
                for unique_ent in unique_ents
            ):
                # print(f"""ent "{ent["ent.text"]}" already in unique_ents""")
                unique_ent_texts = [
                    unique_ent["ent.text"] for unique_ent in unique_ents
                ]
                index_of_same_ent = unique_ent_texts.index(
                    ent["ent.text"]
                )
                unique_ents_count[index_of_same_ent] += 1
            else:
                # print(f"""ent "{ent["ent.text"]}" not already in unique_ents""")
                unique_ents.append(ent)
                unique_ents_count.append(1)        
    return exploded_doc["doc"], unique_ents, unique_ents_count


# Define function for getting ratio of docs where ent appears
def get_ratio(doc, unique_ents, unique_ents_count, all_docs):
    all_doc_texts = [doc.text for doc in all_docs]
    # print(all_doc_texts.count(doc.text))
    unique_ents_proportion = [i / 10 for i in unique_ents_count]
    return doc, unique_ents, unique_ents_proportion


# Define a function for finding frequent annotations (above certain threshold)
def retrieve_freq_or_infreq_ents(
    doc, unique_ents, unique_ents_ratio, threshold=0.5, find_freq=True
):
    if find_freq == True:
        frequent_ents_for_doc = [
            unique_ent["ent"]
            for unique_ent, unique_ent_ratio in zip(unique_ents, unique_ents_ratio)
            if unique_ent_ratio > threshold
        ]
        return doc, frequent_ents_for_doc

    if find_freq == False:
        infrequent_ents_for_doc = [
            unique_ent["ent"]
            for unique_ent, unique_ent_ratio in zip(unique_ents, unique_ents_ratio)
            if unique_ent_ratio < threshold
        ]
        return doc, infrequent_ents_for_doc


# Define a function for deleting any ents in a doc that exist in the same span as a frequent ent
def del_ents_from_freq(doc, frequent_ent_for_doc):
    # Find indexes of doc.ents where either the start- or end character is the same as for the frequent entity
    idxs_of_removable_ents = [
        idx
        for idx, item in enumerate(list(doc.ents))
        if (
            item.start_char == frequent_ent_for_doc.start_char
            or item.end_char == frequent_ent_for_doc.end_char
        )
    ]
    # Remove doc.ents with those indices
    doc_ents = list(doc.ents)
    for idx in sorted(idxs_of_removable_ents, reverse=True):
        del doc_ents[idx]
    doc.ents = tuple(doc_ents)
    return doc


# Define a function for deleting any ents in a doc that exists in any of the same spans as a list of frequent ents
def del_ents_from_freq_multiple(doc, frequent_ents_for_doc):
    for frequent_ent_for_doc in frequent_ents_for_doc:
        doc = del_ents_from_freq(doc, frequent_ent_for_doc)
    return doc


# Define a function for adding a frequent entity to a doc
def add_freq_ent_to_doc(doc, frequent_ent_for_a_doc):
    new_doc_ents = doc.ents + (frequent_ent_for_a_doc,)
    doc.ents = new_doc_ents
    return doc


# Define a function for adding frequent ents in a list of ents to a doc
def add_freq_ents_to_doc(doc, frequent_ents_for_a_doc):
    for frequent_ent_for_a_doc in frequent_ents_for_a_doc:
        doc = add_freq_ent_to_doc(doc, frequent_ent_for_a_doc)
    return doc


# Define a function for finding the index of a list, in which the doc matches another doc
def get_same_doc_index(doc, list_of_docs):
    for i, e in enumerate(list_of_docs):
        if e.text == doc.text:
            return i


# Define a function for streamlining a doc in accordance with frequent_ents_for_doc and infrequent_ents_for_doc
def streamline(rater_doc, infrequent_ents_for_doc, frequent_ents_for_doc):
    r = copy.deepcopy(rater_doc)
    # Delete all entities in the doc that has the same span as the infrequent entities
    r = del_ents_from_freq_multiple(
        r, infrequent_ents_for_doc
    )
    # Delete all entities in the doc that has the same span as the frequent entities
    r = del_ents_from_freq_multiple(
        r, frequent_ents_for_doc
    )
    # Add all frequent entities to the doc
    r = add_freq_ents_to_doc(
        r, frequent_ents_for_doc
    )
    return r


# Defining a function for the entire streamlining

In [451]:
def streamline_doc_for_rater(doc, flat_list, rater_docs, thresholds):
    # Retrieve all annotations across raters for a doc
    ents = retrieve_all_ents(doc, flat_list)

    # Add all entities to doc, and "explode" the doc (dictionary format, with all relevant info)
    exploded_doc = explode_doc(doc, ents)

    # Get a count of all unique ents
    doc, unique_ents_full, unique_ents_count_full = doc_ents_count(exploded_doc, match = 'label_and_text')

    # Get the ratio of occurrence of all unique entities
    doc, unique_ents_full, unique_ents_ratio_full = get_ratio(
        doc, unique_ents_full, unique_ents_count_full, flat_list
    )

    # Retrieve the entities that are frequent across all raters
    doc, frequent_ents_for_doc = retrieve_freq_or_infreq_ents(
        doc, unique_ents_full, unique_ents_ratio_full, threshold=thresholds['find_freq'], find_freq=True
    )

    # Get a count of all unique ents
    doc, unique_ents_partial, unique_ents_count_partial = doc_ents_count(exploded_doc, match = 'text')

    # Get the ratio of occurrence of all unique entities
    doc, unique_ents_partial, unique_ents_ratio_partial = get_ratio(
        doc, unique_ents_partial, unique_ents_count_partial, flat_list
    )

    # Retrieve the entities that are infrequent across all raters
    doc, infrequent_ents_for_doc = retrieve_freq_or_infreq_ents(
        doc, unique_ents_partial, unique_ents_ratio_partial, threshold=thresholds['find_infreq'], find_freq=False
    )
    
    unique_ents_full_texts = [ent['ent.text'] for ent in unique_ents_full]
    unique_ents_partial_texts = [ent['ent.text'] for ent in unique_ents_partial]
    
    print(f'Doc: {doc.text} \nunique_ents_full: {unique_ents_full_texts} \nunique_ents_ratio_full: {unique_ents_ratio_full} \nfrequent_ents: {frequent_ents_for_doc} \nunique_ents_partial: {unique_ents_partial_texts} \nunique_ents_ratio_partial: {unique_ents_ratio_partial} \ninfrequent_ents: {infrequent_ents_for_doc}')

    # Get index of the doc in question
    idx = get_same_doc_index(doc, rater_docs)

    # If the doc exists in the raters data
    if idx is not None:
        # Retrieve the doc that should be streamlined
        rater_doc = copy.deepcopy(rater_docs[idx])
        print(f'doc.ents BEFORE streamlining: {rater_doc.ents}')
        # Streamline the doc
        streamlined_rater_doc = streamline(rater_doc, infrequent_ents_for_doc, frequent_ents_for_doc)
        print(f'doc.ents AFTER streamlining: {streamlined_rater_doc.ents} \n\n\n\n')
        return streamlined_rater_doc
    
    else:
        print("\n\n\n\ndoc not in rater_docs")

# Loading data

In [452]:
# Change cwd
os.chdir("/Users/emiltrencknerjessen/Desktop/priv/DANSK-gold-NER")

# Load language object
nlp = dacy.load("medium")

# List relevant data and sort by rater number
data_paths = glob.glob("./data/DANSK-multi/unprocessed/rater*/data.spacy")
data_paths.sort()
data_paths.sort(key="./data/DANSK-multi/unprocessed/rater_10/data.spacy".__eq__)

# Load in data and get rater indices (if not already loaded)
data = []
raters = []
for path in data_paths:
    # Get rater indices
    rater = re.search(r"\d+", path).group()
    raters.append(int(rater) - 1)

    # Load data
    doc_bin = DocBin().from_disk(path)
    docs = list(doc_bin.get_docs(nlp.vocab))[:20]
    data.append(docs)

/Users/emiltrencknerjessen/miniconda3/envs/thesis/lib/python3.10/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'da_dacy_medium_trf' (0.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.2.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/Users/emiltrencknerjessen/miniconda3/envs/thesis/lib/python3.10/site-packages/spacy_transformers/pipeline_component.py:406: UserWarning: Automatically converting a transformer component from spacy-transformers v1.0 to v1.1+. If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spacy-transformers version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


# Retrieve unique documents


In [453]:
# Get a list of all unique docs
unique_docs = []
flat_list = [item for sublist in data for item in sublist]
for doc in flat_list:
    if all(doc.text != unique_doc.text for unique_doc in unique_docs):
        unique_docs.append(doc)

# Streamlining a single doc for a single rater

### Getting all necessary variables

In [454]:
# Have a doc and a list of all docs (with duplicates from each rater)
doc = unique_docs[0]
flat_list = flat_list
rater_docs = copy.deepcopy(data[3])
thresholds = {'find_freq': .3, 'find_infreq': .3}

In [455]:
# A single doc for a single rater
streamlined_rater_doc = streamline_doc_for_rater(doc, flat_list, rater_docs, thresholds)

Doc: Hvordan kan statsministeren kalde børn,der er født på danske hospitaler for indvandrerdrenge! 
unique_ents_full: ['danske', 'statsministeren', 'børn', 'hospitaler', 'indvandrerdrenge', 'hospitaler', 'indvandrerdrenge'] 
unique_ents_ratio_full: [1.0, 0.4, 0.2, 0.2, 0.3, 0.2, 0.1] 
frequent_ents: [danske, statsministeren] 
unique_ents_partial: ['danske', 'statsministeren', 'børn', 'hospitaler', 'indvandrerdrenge'] 
unique_ents_ratio_partial: [1.0, 0.4, 0.2, 0.4, 0.4] 
infrequent_ents: [børn]
doc.ents BEFORE streamlining: (statsministeren, danske, hospitaler, indvandrerdrenge)
doc.ents AFTER streamlining: (statsministeren, danske, hospitaler, indvandrerdrenge) 






In [456]:
# All docs for a single rater
streamlined_docs = []
for doc in unique_docs:
    streamlined_docs.append(streamline_doc_for_rater(doc, flat_list, rater_docs, thresholds))

Doc: Hvordan kan statsministeren kalde børn,der er født på danske hospitaler for indvandrerdrenge! 
unique_ents_full: ['danske', 'statsministeren', 'børn', 'hospitaler', 'indvandrerdrenge', 'hospitaler', 'indvandrerdrenge'] 
unique_ents_ratio_full: [1.0, 0.4, 0.2, 0.2, 0.3, 0.2, 0.1] 
frequent_ents: [danske, statsministeren] 
unique_ents_partial: ['danske', 'statsministeren', 'børn', 'hospitaler', 'indvandrerdrenge'] 
unique_ents_ratio_partial: [1.0, 0.4, 0.2, 0.4, 0.4] 
infrequent_ents: [børn]
doc.ents BEFORE streamlining: (statsministeren, danske, hospitaler, indvandrerdrenge)
doc.ents AFTER streamlining: (statsministeren, danske, hospitaler, indvandrerdrenge) 




Doc: Det her er jo håbløst #dkpol  https://t.co/e7hAg155Gw 
unique_ents_full: ['dkpol', '#dkpol', 'dkpol'] 
unique_ents_ratio_full: [0.1, 0.1, 0.1] 
frequent_ents: [] 
unique_ents_partial: ['dkpol', '#dkpol'] 
unique_ents_ratio_partial: [0.2, 0.1] 
infrequent_ents: [dkpol, #dkpol]
doc.ents BEFORE streamlining: ()
doc.ents 

In [460]:
# All docs for all raters
flat_list = flat_list
thresholds = {'find_freq': .3, 'find_infreq': .3}

streamlined_data = []
for rater in raters:
    
    streamlined_docs = []
    rater_docs = copy.deepcopy(data[3])
    for doc in unique_docs:
        print(f'Current rater:{rater}')
        streamlined_docs.append(streamline_doc_for_rater(doc, flat_list, rater_docs, thresholds))
    streamlined_data.append(streamlined_docs)

Current rater:0

Doc: Hvordan kan statsministeren kalde børn,der er født på danske hospitaler for indvandrerdrenge! 
unique_ents_full: ['danske', 'statsministeren', 'børn', 'hospitaler', 'indvandrerdrenge', 'hospitaler', 'indvandrerdrenge'] 
unique_ents_ratio_full: [1.0, 0.4, 0.2, 0.2, 0.3, 0.2, 0.1] 
frequent_ents: [danske, statsministeren] 
unique_ents_partial: ['danske', 'statsministeren', 'børn', 'hospitaler', 'indvandrerdrenge'] 
unique_ents_ratio_partial: [1.0, 0.4, 0.2, 0.4, 0.4] 
infrequent_ents: [børn]
doc.ents BEFORE streamlining: (statsministeren, danske, hospitaler, indvandrerdrenge)
doc.ents AFTER streamlining: (statsministeren, danske, hospitaler, indvandrerdrenge) 




Current rater:0

Doc: Det her er jo håbløst #dkpol  https://t.co/e7hAg155Gw 
unique_ents_full: ['dkpol', '#dkpol', 'dkpol'] 
unique_ents_ratio_full: [0.1, 0.1, 0.1] 
frequent_ents: [] 
unique_ents_partial: ['dkpol', '#dkpol'] 
unique_ents_ratio_partial: [0.2, 0.1] 
infrequent_ents: [dkpol, #dkpol]
doc.ents

KeyboardInterrupt: 

In [ ]:
# # Save all streamlined docs as DocBins
# for rater in raters:
#     db = DocBin()
#     savepath = f"./data/DANSK-multi/streamlined/rater_{rater+1}"
#     for doc in data[rater]:
#         db.add(doc)
#     db.to_disk(savepath)
